In [1]:
import glob
import pandas as pd
from tqdm import tqdm
from itertools import combinations
from fuzzywuzzy import fuzz, process
import numpy as np

In [7]:
# Get all paths to CSV files in the specified directory
csv_files = glob.glob('/home/anjah/Documents/mag/BD/project/BD_project/data/NYTickets/*.csv')

# Print the number of CSV files found
print("Number of CSV files:", len(csv_files))


Number of CSV files: 11


In [ ]:
#read files and get unique values for combination of Street and Violation County 
#save them to csv

In [10]:
# Create an empty DataFrame
df = pd.DataFrame()

# Define chunk size (number of rows per chunk)
chunk_size = 10000  # Adjust based on your available memory

# Process each file with a progress bar
for file in tqdm(csv_files, desc="Processing files"):
    # Read the CSV file in chunks
    for chunk in pd.read_csv(file, chunksize=chunk_size):
        # Rename columns to have a consistent name
        if 'Street Name' in chunk.columns:
            chunk = chunk.rename(columns={'Street Name': 'Street'})
        
        # Check if the required columns exist in the chunk
        if 'Street' in chunk.columns and 'Violation County' in chunk.columns:
            # Get the unique values for the combination of Street and Violation County
            temp_df = chunk[['Street', 'Violation County']].drop_duplicates()
            
            # Append the unique values to the DataFrame
            df = pd.concat([df, temp_df]).drop_duplicates().reset_index(drop=True)
        else:
            # Skipping chunk without printing a message
            pass

# Optionally, reset the index of the final DataFrame
df.reset_index(drop=True, inplace=True)

# Check the final DataFrame
print(f"Final DataFrame has {len(df)} unique rows")

Processing files: 100%|██████████| 11/11 [32:56<00:00, 179.69s/it]

Final DataFrame has 467005 unique rows


In [16]:
#save df to csv file
df.to_csv('/home/anjah/Documents/mag/BD/project/BD_project/data/NYTickets/unique_street_county.csv', index=False)

In [21]:
df.head()

,Street,Violation County
0,MACDOUNGH ST,K
1,LEXINGTON AVE,K
2,FULTON ST,K
3,MADISON ST,K
4,HALSEY ST,K


In [15]:
df.dropna(inplace=True)

In [19]:
len(df)

419105

In [20]:
df.loc[8, 'Street']

KeyError: 8

In [34]:
# Function to find similar street names within the same Violation County
def find_similar_streets(group):
    similar_streets = {}
    for idx, row in group.iterrows():
        matches = process.extract(row['Street'], group['Street'], scorer=fuzz.token_sort_ratio)
        for match, score, _ in matches:
            if match != row['Street'] and score >= 80:  # Adjust threshold as needed
                if row['Street'] in similar_streets:
                    similar_streets[row['Street']].append(match)
                else:
                    similar_streets[row['Street']] = [match]
    return similar_streets

# Apply the function within each group of Violation County
tqdm.pandas(desc="Finding similar streets")
similarities = df.groupby('Violation County').progress_apply(find_similar_streets)

# Combine results from all groups
corresponding_streets = {}
for similar_streets in similarities:
    for street, matches in similar_streets.items():
        if street in corresponding_streets:
            corresponding_streets[street].extend(matches)
        else:
            corresponding_streets[street] = matches

# Print results
for street, matches in corresponding_streets.items():
    print(f"{street}: {', '.join(matches)}")

Finding similar streets:  20%|██        | 8/40 [16:15<1:05:02, 121.95s/it]


KeyboardInterrupt: 

In [ ]:
#read /home/anjah/Documents/mag/BD/project/BD_project/data/augmented/2014_augmented.parquet and transform it into a csv file

In [2]:
#get all filepaths in /home/anjah/Documents/mag/BD/project/BD_project/data/augmented directory
filepaths = glob.glob('/home/anjah/Documents/mag/BD/project/BD_project/data/augmented/*.parquet')

In [ ]:
for filepath in filepaths:
    # Read the augmented data
    print(f"Reading {filepath}")
    augmented_data = pd.read_parquet(filepath)

    # Save the augmented data to a CSV file
    augmented_data.to_csv(filepath.replace('.parquet', '.csv'), index=False)

Reading /home/anjah/Documents/mag/BD/project/BD_project/data/augmented/2020_augmented.parquet


In [1]:
from collections import defaultdict

In [2]:
count_tickets_day = defaultdict(float)

In [7]:
count_tickets_day['Monday'] += 1
count_tickets_day['Tuesday'] += 5

In [8]:
count_tickets_day

defaultdict(float, {'Monday': 3.0, 'Tuesday': 10.0})

In [24]:
import datetime
import fiscalyear
from fiscalyear import *
import math

In [10]:
now = datetime.datetime.now()
#get fiscal week
fiscal_week = now.strftime("%V")

In [11]:
fiscal_week

'32'

In [14]:
fiscalyear.setup_fiscal_calendar(start_month=7)

In [20]:
e = FiscalDateTime.now()

In [26]:
e = FiscalDateTime(now)

TypeError: an integer is required (got type datetime.datetime)

In [25]:
#round up 
math.ceil(e.fiscal_day/7)

6

In [18]:
FiscalYear(2014).start

FiscalDateTime(2013, 7, 1, 0, 0)

In [19]:
#get the fiscal week of today
fiscal_week = FiscalYear(2014).week(now)

AttributeError: 'FiscalYear' object has no attribute 'week'

In [3]:
012+12


SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (644555677.py, line 1)

In [9]:
from collections import defaultdict

# Create a defaultdict where each value is another defaultdict for boroughs
day_borough_counts = defaultdict(lambda: defaultdict(int))

# Function to update counts
def update_count(day, borough, count):
    day_borough_counts[day][borough] += count

# Example usage:
# Assume 'day' is an integer where 0 = Monday, 1 = Tuesday, ..., 6 = Sunday
update_count(0, 'Manhattan', 1)  # Increment Monday count for Manhattan
update_count(2, 'Brooklyn', 3)   # Increment Wednesday count for Brooklyn by 3

# To access the counts:
for day, borough_counts in day_borough_counts.items():
    print(f"Day {day}: {dict(borough_counts)}")


Day 0: {'Manhattan': 1}
Day 2: {'Brooklyn': 3}


In [18]:
from collections import defaultdict
import statistics

# Create a defaultdict to store lists of counts for each borough on each day
weekly_counts = defaultdict(lambda: defaultdict(int))

# Function to update weekly counts
def update_weekly_count(week_number, day, borough, count):
    # Add the count to the list for the given borough on the specified day
    weekly_counts[day][borough]+= count

# Example usage
# Assume 'day' is a string representing the day of the week (e.g., "Monday", "Tuesday", etc.)
for i in range(1, 8):
    update_weekly_count(1, 'Monday', 'Manhattan', i)
    update_weekly_count(1, 'Monday', 'Brooklyn', i)
    update_weekly_count(1, 'Tuesday', 'Manhattan', i)
    update_weekly_count(1, 'Tuesday', 'Brooklyn', i)

In [19]:
weekly_counts

defaultdict(<function __main__.<lambda>()>,
            {'Monday': defaultdict(int, {'Manhattan': 28, 'Brooklyn': 28}),
             'Tuesday': defaultdict(int, {'Manhattan': 28, 'Brooklyn': 28})})

In [16]:
weekly_counts['Monday']['Manhattan']

[1, 2, 3, 4, 5, 6, 7]